# Table

In [ ]:
file_path = '../results/result_mid3.csv'

In [ ]:
import pandas as pd
from IPython.display import display

df = pd.read_csv(file_path)
display(df)

# T-SNE

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from adjustText import adjust_text

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

test_data = pd.read_csv('../../../../mnt/sda/suhohan/emnist/emnist-byclass-test.csv')
num_test_samples = test_data.shape[0]

x_test = test_data.iloc[:, 1:].to_numpy().reshape((num_test_samples, 28, 28, 1))
y_test = tf.keras.utils.to_categorical(test_data.iloc[:, 0], 62)
y_test_int = test_data.iloc[:, 0].to_numpy()

x_test_corrected = np.array([np.fliplr(np.rot90(img.reshape(28, 28), k=3))
                            for img in x_test]).reshape(num_test_samples, 28, 28, 1)

In [ ]:
def get_latest_model_file(checkpoint_dir):
    files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.hdf5')]
    files.sort(key=lambda x: os.path.getmtime(os.path.join(checkpoint_dir, x)), reverse=True)
    return os.path.join(checkpoint_dir, files[0]) if files else None

In [ ]:
checkpoint_dir = './checkpoints/checkpoints_final/'

latest_model_file = get_latest_model_file(checkpoint_dir)
if latest_model_file:
    print(f"Loading model from: {latest_model_file}")
    model = tf.keras.models.load_model(latest_model_file)
    loss_ours, acc_ours = model.evaluate(x_test, y_test_int)
    print("Loss:", loss_ours)
    print("Accuracy:", acc_ours)
    predictions = model.predict(x_test_corrected)
else:
    print("No model files found in the checkpoint directory.")
    predictions = None

In [ ]:
from sklearn.manifold import TSNE

tsne_result_path = 'visualization/tsne_results_ours.csv'

if predictions is not None:
    if os.path.exists(tsne_result_path):
        images_tsne = pd.read_csv(tsne_result_path).to_numpy()
    else:
        # t-SNE 적용
        tsne = TSNE(n_components=2, random_state=42)
        images_tsne = tsne.fit_transform(predictions)
        # t-SNE 결과 저장
        np.savetxt(tsne_result_path, images_tsne, delimiter=',')
else:
    print("No predictions to apply t-SNE.")

In [ ]:
def label_transform(label):
    if 0 <= label <= 9:
        return str(label)
    elif 10 <= label <= 35:
        return chr(label + ord('a') - 10)
    elif 36 <= label <= 61:
        return chr(label + ord('A') - 36)
    else:
        return str(label)


sample_images = x_test.reshape((num_test_samples, -1))
sample_labels = test_data.iloc[:, 0]

transformed_labels = sample_labels.apply(label_transform)

In [ ]:
if predictions is not None:
    np.random.seed(42)
    random_colors = np.random.rand(62, 3)
    cmap = ListedColormap(random_colors)
    plt.figure(figsize=(12, 10))

    if y_test_int.ndim == 1:
        sample_labels = y_test_int
    else:
        sample_labels = np.argmax(y_test_int, axis=1)

    # Ensure images_tsne and sample_labels have the same length
    if len(images_tsne) == len(sample_labels):
        scatter = plt.scatter(images_tsne[:, 0], images_tsne[:, 1], c=sample_labels, cmap=cmap, s=3, alpha=0.5)
        colorbar = plt.colorbar(scatter)

        colorbar.set_ticks(np.arange(62))
        colorbar.set_ticklabels([label_transform(i) for i in range(62)])
        colorbar.ax.tick_params(labelsize=8)

        texts = []
        for i in range(62):
            mask = sample_labels == i
            mean_x = np.mean(images_tsne[mask, 0])
            mean_y = np.mean(images_tsne[mask, 1])
            label = label_transform(i)
            color = random_colors[i]
            text_color = 'white' if np.mean(color) < 0.5 else 'black'
            texts.append(plt.text(mean_x, mean_y, label, fontsize=9, ha='center', va='center',
                                  bbox=dict(facecolor=color, edgecolor='none', boxstyle='round,pad=0.3'), color=text_color))

        adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray', lw=0.7))

        plt.title('t-SNE visualization of EMNIST ByClass subset')
        plt.xlabel('t-SNE feature 1')
        plt.ylabel('t-SNE feature 2')
        plt.savefig('tsne_visualization_ours.png', dpi=300)
        plt.show()
    else:
        print(
            f"Size mismatch: images_tsne has {len(images_tsne)} elements, but sample_labels has {len(sample_labels)} elements.")
else:
    print("No t-SNE results to visualize.")

In [3]:
import pandas as pd
from IPython.display import display

file_path = '../results/result_optimizer.csv'

df = pd.read_csv(file_path)

display(df)

,Optimizer,Loss,Accuracy,Training Time
0,SGD,0.339564,0.871667,1277.049462
1,SGD,0.333832,0.873842,1286.936976
2,Adagrad,0.377922,0.862064,1289.939265
3,RMSprop,0.313961,0.882645,503.086377
4,Adam,0.308624,0.883504,605.063532
5,Adamax,0.306209,0.884316,775.569509
6,Nadam,0.308206,0.884355,784.482589
